In [1]:
import csv
import math

file='/Users/tej/Desktop/labs/ml/lab1/advertising.csv'
data=[]
with open(file,'r') as f:
    reader=csv.reader(f)
    header=next(reader)
    for row in reader:
        data.append([float(x) if x!='' else 0 for x in row])

In [2]:
def scale(data):
    rows, cols = len(data), len(data[0])
    scaled = [[0]*cols for _ in range(rows)]
    for j in range(cols):
        col = [data[i][j] for i in range(rows)]
        mean = sum(col) / rows
        std = math.sqrt(sum((x - mean)**2 for x in col) / rows)
        for i in range(rows):
            scaled[i][j] = (data[i][j] - mean) / std
    return scaled

In [3]:
def train_test_split(X, y, test_ratio=0.2):
    split = int(len(X) * (1 - test_ratio))
    return X[:split], X[split:], y[:split], y[split:]

In [4]:
def remove_outliers(data):
    cols = len(data[0])
    keep_rows = [True] * len(data)
    for j in range(cols):
        col = [row[j] for row in data]
        Q1 = percentile(col, 25)
        Q3 = percentile(col, 75)
        IQR = Q3 - Q1
        for i in range(len(col)):
            if col[i] < Q1 - 1.5 * IQR or col[i] > Q3 + 1.5 * IQR:
                keep_rows[i] = False
    return [row for i, row in enumerate(data) if keep_rows[i]]

def percentile(values, percent):
    values = sorted(values)
    k = (len(values)-1) * percent/100
    f = math.floor(k)
    c = math.ceil(k)
    if f == c:
        return values[int(k)]
    return values[f] + (values[c] - values[f]) * (k - f)

In [5]:
def predict(X, weights, bias):
    preds = []
    for row in X:
        pred = sum(row[j] * weights[j] for j in range(len(row))) + bias
        preds.append(pred)
    return preds

In [6]:
def gradient_descent(X, y, lr=0.01, epochs=1000):
    rows, cols = len(X), len(X[0])
    weights = [0.0] * cols
    bias = 0.0

    for _ in range(epochs):
        y_pred = predict(X, weights, bias)
        dw = [0.0] * cols
        db = 0.0
        for i in range(rows):
            error = y_pred[i] - y[i]
            for j in range(cols):
                dw[j] += (error * X[i][j]) / rows
            db += error / rows
        for j in range(cols):
            weights[j] -= lr * dw[j]
        bias -= lr * db
    return weights, bias

In [7]:
def r2_score(y_true, y_pred):
    mean_y = sum(y_true) / len(y_true)
    ss_tot = sum((y - mean_y) ** 2 for y in y_true)
    ss_res = sum((y_true[i] - y_pred[i]) ** 2 for i in range(len(y_true)))
    return 1 - (ss_res / ss_tot)

In [8]:
X = [row[:-1] for row in data]
y = [row[-1] for row in data]

In [9]:
X_scaled=scale(X)
y_mean = sum(y) / len(y)
y_std = math.sqrt(sum((yi - y_mean) ** 2 for yi in y) / len(y))
y_scaled = [(yi - y_mean) / y_std for yi in y]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [11]:
theta,b=gradient_descent(X_train,y_train)

In [12]:
y_pred =predict(X_test,theta,b)

In [13]:
r2_bef=r2_score(y_test,y_pred)

In [14]:
no_outliers=remove_outliers(data)
X_no=[row[:-1] for row in no_outliers]
y_no=[row[-1] for row in no_outliers]

In [15]:
X_scaled_no=scale(X_no)
y_mean_no = sum(y_no) / len(y_no)
y_std_no = math.sqrt(sum((yi - y_mean_no) ** 2 for yi in y_no) / len(y_no))
y_scaled_no = [(yi - y_mean_no) / y_std_no for yi in y_no]

In [16]:
X_train_no, X_test_no, y_train_no, y_test_no = train_test_split(X_scaled_no, y_scaled_no)

In [17]:
theta_no,b_no=gradient_descent(X_train_no,y_train_no)

In [18]:
y_pred_no =predict(X_test_no,theta_no,b_no)

In [19]:
r2_after=r2_score(y_test_no,y_pred_no)

In [20]:
print("before:",r2_bef)
print("after:",r2_after)

before: 0.902446534309072
after: 0.9020702618376557


-----------------------------------2nd question----------------------------------------------------------

In [22]:
import pandas as pd

In [23]:
df_av = pd.read_csv('faa_ai_prelim.csv')

In [80]:
df_av

,UPDATED,ENTRY_DATE,EVENT_LCL_DATE,EVENT_LCL_TIME,LOC_CITY_NAME,LOC_STATE_NAME,LOC_CNTRY_NAME,RMK_TEXT,EVENT_TYPE_DESC,FSDO_DESC,...,PAX_INJ_MINOR,PAX_INJ_SERIOUS,PAX_INJ_FATAL,PAX_INJ_UNK,GRND_INJ_NONE,GRND_INJ_MINOR,GRND_INJ_SERIOUS,GRND_INJ_FATAL,GRND_INJ_UNK,EVENT_DATETIME
0,No,19-FEB-16,19-FEB-16,00:45:00,MARSHVILLE,North Carolina,NaN,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",Accident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-19 00:45:00
1,No,19-FEB-16,18-FEB-16,23:55:00,TAVERNIER,Florida,NaN,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,Incident,FAA Miami FSDO-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 23:55:00
2,No,19-FEB-16,18-FEB-16,22:14:00,TRENTON,New Jersey,NaN,"AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...",Incident,FAA Philadelphia FSDO-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 22:14:00
3,No,19-FEB-16,18-FEB-16,17:10:00,ASHEVILLE,North Carolina,NaN,"AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...",Incident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 17:10:00
4,No,19-FEB-16,18-FEB-16,00:26:00,TALKEETNA,Alaska,NaN,"AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...",Incident,FAA Anchorage FSDO-03,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 00:26:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,No,08-FEB-16,31-DEC-15,17:00:00,SAN ANTONIO,Texas,NaN,"AIRCRAFT ON LANDING, GROUND LOOPED, BULVERDE A...",Accident,FAA San Antonio FSDO-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31 17:00:00
79,No,08-FEB-16,05-FEB-16,11:17:00,MARICOPA,Arizona,NaN,"AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...",Accident,FAA Scottsdale FSDO-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05 11:17:00
80,No,08-FEB-16,05-FEB-16,23:02:00,SAN PEDRO,California,NaN,"N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...",Accident,FAA Long Beach FSDO-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05 23:02:00
81,No,08-FEB-16,05-FEB-16,23:02:00,SAN PEDRO,California,NaN,"N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...",Accident,FAA Long Beach FSDO-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05 23:02:00


In [24]:
df_av['EVENT_LCL_TIME'] = df_av['EVENT_LCL_TIME'].str.replace('Z', '', regex=False)
df_av['EVENT_DATETIME'] = pd.to_datetime(df_av['EVENT_LCL_DATE'] + ' ' + df_av['EVENT_LCL_TIME'],errors='coerce', format='%d-%b-%y %H:%M:%S')

In [25]:
columns_needed = [
    'EVENT_DATETIME',
    'ACFT_MAKE_NAME',
    'LOC_STATE_NAME',
    'ACFT_MODEL_NAME',
    'RMK_TEXT',
    'FLT_PHASE',
    'EVENT_TYPE_DESC',
    'FATAL_FLAG',
    'ACFT_DMG_DESC'
]
new_df = df_av[columns_needed].copy()

In [26]:
new_df.shape

(83, 9)

In [27]:
new_df.info()
new_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   EVENT_DATETIME   83 non-null     datetime64[ns]
 1   ACFT_MAKE_NAME   78 non-null     object        
 2   LOC_STATE_NAME   83 non-null     object        
 3   ACFT_MODEL_NAME  79 non-null     object        
 4   RMK_TEXT         83 non-null     object        
 5   FLT_PHASE        82 non-null     object        
 6   EVENT_TYPE_DESC  83 non-null     object        
 7   FATAL_FLAG       8 non-null      object        
 8   ACFT_DMG_DESC    74 non-null     object        
dtypes: datetime64[ns](1), object(8)
memory usage: 6.0+ KB


,EVENT_DATETIME,ACFT_MAKE_NAME,LOC_STATE_NAME,ACFT_MODEL_NAME,RMK_TEXT,FLT_PHASE,EVENT_TYPE_DESC,FATAL_FLAG,ACFT_DMG_DESC
0,2016-02-19 00:45:00,BEECH,North Carolina,36,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",UNKNOWN (UNK),Accident,Yes,Unknown
1,2016-02-18 23:55:00,VANS,Florida,RV7,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,LANDING (LDG),Incident,NaN,Unknown
2,2016-02-18 22:14:00,CESSNA,New Jersey,172,"AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...",APPROACH (APR),Incident,NaN,Minor
3,2016-02-18 17:10:00,LANCAIR,North Carolina,235,"AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...",LANDING (LDG),Incident,NaN,Unknown
4,2016-02-18 00:26:00,CESSNA,Alaska,172,"AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...",LANDING (LDG),Incident,NaN,Minor


In [28]:
new_df['FATAL_FLAG'] = new_df['FATAL_FLAG'].fillna('No')

In [29]:
fill = new_df['FLT_PHASE'].mode()
mode = fill[0]
new_df['FLT_PHASE'] = new_df['FLT_PHASE'].fillna(mode)
fill1 = new_df['ACFT_DMG_DESC'].mode()
mode1 = fill1[0]
new_df['ACFT_DMG_DESC'] = new_df['ACFT_DMG_DESC'].fillna(mode1)

In [30]:
new_df.dropna(subset=['ACFT_MAKE_NAME'], inplace=True)
new_df.dropna(subset=['ACFT_MODEL_NAME'], inplace=True)

In [31]:
new_df.isnull().sum()

EVENT_DATETIME     0
ACFT_MAKE_NAME     0
LOC_STATE_NAME     0
ACFT_MODEL_NAME    0
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
ACFT_DMG_DESC      0
dtype: int64

In [32]:
new_df.shape

(77, 9)

In [33]:
airplane_name = new_df['ACFT_MAKE_NAME'].value_counts()
print(airplane_name)

ACFT_MAKE_NAME
CESSNA                    23
PIPER                     10
BEECH                      9
MOONEY                     4
BOEING                     3
BELL                       2
LANCAIR                    2
SOCATA                     2
CHAMPION                   2
AERONCA                    1
SABRELINER                 1
PITTS                      1
FLIGHT DESIGN              1
GULFSTREAM                 1
GRUMMAN                    1
AIRBUS                     1
AEROSTAR INTERNATIONAL     1
AERO COMMANDER             1
SAAB                       1
MAULE                      1
EMBRAER                    1
VANS                       1
FAIRCHILD                  1
CONSOLIDATED VULTEE        1
GREAT LAKES                1
GLOBE                      1
HUGHES                     1
CHRISTEN                   1
NORTH AMERICAN             1
Name: count, dtype: int64


In [34]:
fatal_obs = new_df[new_df['FATAL_FLAG'] == 'Yes']
print(fatal_obs)
print(len(fatal_obs))


        EVENT_DATETIME  ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0  2016-02-19 00:45:00           BEECH  North Carolina              36   
53 2016-02-11 11:32:00           PIPER         Florida            PA28   
55 2016-02-11 00:17:00   FLIGHT DESIGN      California            CTLS   
79 2016-02-05 11:17:00  NORTH AMERICAN         Arizona             F51   
80 2016-02-05 23:02:00        CHAMPION      California           8KCAB   
81 2016-02-05 23:02:00           BEECH      California              35   
82 2016-02-02 01:52:00          CESSNA         Alabama             182   

                                             RMK_TEXT      FLT_PHASE  \
0   AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...  UNKNOWN (UNK)   
53  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...  UNKNOWN (UNK)   
55  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...  UNKNOWN (UNK)   
79  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...  UNKNOWN (UNK)   
80  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BEL

In [35]:
unique_cats = new_df['ACFT_DMG_DESC'].unique()
print(unique_cats)

['Unknown' 'Minor' 'Substantial' 'Destroyed']


In [36]:
for cat in unique_cats:
    col_name = f"DMG_{cat}"
    new_df[col_name] = (new_df['ACFT_DMG_DESC'] == cat).astype(int)


In [37]:
new_df.drop(columns=[f"DMG_Unknown"], inplace=True)

In [38]:
new_df.sample()

,EVENT_DATETIME,ACFT_MAKE_NAME,LOC_STATE_NAME,ACFT_MODEL_NAME,RMK_TEXT,FLT_PHASE,EVENT_TYPE_DESC,FATAL_FLAG,ACFT_DMG_DESC,DMG_Minor,DMG_Substantial,DMG_Destroyed
49,2016-02-16 02:00:00,GRUMMAN,California,AA1,"AIRCRAFT ON LANDING NOSE GEAR COLLAPSED, SALIN...",LANDING (LDG),Incident,No,Minor,1,0,0


In [39]:
def extract_phase(text):
    if pd.isnull(text):
        return 'UNKNOWN'
    phases = ['LANDING', 'TAKEOFF', 'CRUISE', 'APPROACH']
    for phase in phases:
        if phase in text.upper():
            return phase
    return 'UNKNOWN'

new_df['FLIGHT_PHASE_TEXT'] = new_df['RMK_TEXT'].apply(extract_phase)


In [40]:
comparison = pd.DataFrame({
    'Original_Phase': new_df['FLT_PHASE'],
    'Text_Phase': new_df['FLIGHT_PHASE_TEXT']
})
print(comparison.head(20))


    Original_Phase Text_Phase
0    UNKNOWN (UNK)    UNKNOWN
1    LANDING (LDG)    LANDING
2   APPROACH (APR)    UNKNOWN
3    LANDING (LDG)    LANDING
4    LANDING (LDG)    LANDING
5    UNKNOWN (UNK)    UNKNOWN
6    LANDING (LDG)    LANDING
7    LANDING (LDG)    LANDING
8    TAKEOFF (TOF)    UNKNOWN
10   LANDING (LDG)    LANDING
12   LANDING (LDG)    LANDING
13   LANDING (LDG)    LANDING
14   TAKEOFF (TOF)    TAKEOFF
15   LANDING (LDG)    UNKNOWN
16   TAKEOFF (TOF)    TAKEOFF
17   LANDING (LDG)    LANDING
18   TAKEOFF (TOF)    UNKNOWN
19   LANDING (LDG)    LANDING
20   LANDING (LDG)    LANDING
21   UNKNOWN (UNK)    UNKNOWN
